In [4]:
# base class for all nural network modules,
# UDF modules should inherit this class
# allowing nest modules, a module includes other modules
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.models as models

In [2]:
print(torch.__version__)

1.10.2


In [11]:

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
# add_module
my_model = Model()
# Adds a child module to the current module.
# The module can be accessed as an attribute using the given name.
my_model.add_module('name',module)

# Applies fn recursively to every submodule (as returned by
# .children()) as well as self. Typical use includes initializing
# the parameters of a model (see also torch.nn.init).
def fn(weights):
    pass
my_model.apply(fn)

TypeError: add_module() missing 2 required positional arguments: 'name' and 'module'

In [13]:
@torch.no_grad()
def init_weights(m):
    print(m)
    if type(m) == nn.Linear:
        m.weight.fill_(1.0)
        print(m.weight)
    print('type:',type(m))

net = nn.Sequential(nn.Linear(2, 2), nn.Linear(2, 2))
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
net.apply(init_weights)

Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
type: <class 'torch.nn.modules.linear.Linear'>
Linear(in_features=2, out_features=2, bias=True)
Parameter containing:
tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
type: <class 'torch.nn.modules.linear.Linear'>
Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)
type: <class 'torch.nn.modules.container.Sequential'>


Sequential(
  (0): Linear(in_features=2, out_features=2, bias=True)
  (1): Linear(in_features=2, out_features=2, bias=True)
)

In [37]:
# parameter vs buffer
# parameter: calculate gradient
# buffer: do not compute gradient
for buf in my_model.buffers():
    print(type(buf), buf.size())

for param in my_model.parameters():
    print(param)

Parameter containing:
tensor([[[[ 0.1555, -0.1904,  0.1007, -0.1543,  0.0335],
          [ 0.1126,  0.0402, -0.1774, -0.0415, -0.1666],
          [-0.1572,  0.1696, -0.0159, -0.1752,  0.0927],
          [ 0.0245, -0.0808, -0.0441,  0.1695,  0.0675],
          [-0.1693, -0.1084,  0.1196,  0.1757, -0.1406]]],


        [[[-0.0321, -0.0956, -0.1140, -0.0086, -0.0618],
          [ 0.0343,  0.0722,  0.0246, -0.0106, -0.0427],
          [-0.0138, -0.1679,  0.0410,  0.1437, -0.1461],
          [ 0.1242,  0.0710,  0.1457,  0.1823,  0.0951],
          [ 0.1734, -0.0379,  0.1789,  0.0467,  0.0278]]],


        [[[-0.0385, -0.0345,  0.0564,  0.0888, -0.0266],
          [ 0.1788,  0.0678, -0.0802,  0.1659,  0.0532],
          [ 0.1465,  0.1191, -0.1121, -0.1802,  0.1661],
          [-0.1589,  0.0829, -0.0032,  0.0950,  0.0521],
          [-0.1870,  0.0664,  0.0513,  0.1224,  0.0263]]],


        [[[ 0.1451,  0.1351, -0.1409,  0.1106,  0.1012],
          [-0.1257, -0.0584, -0.1194, -0.0503,  0.1520

net.children()#Returns an iterator over immediate children modules.
# This method modifies the module in-place.
net.cpu()# Moves all model parameters and buffers to the CPU.

# Moves all model parameters and buffers to the GPU.
# This also makes associated parameters and buffers different objects.
# So it should be called before constructing optimizer if the module
# will live on GPU while being optimized.
net.cuda()

# Sets the module in evaluation mode. This has any effect only on
# certain modules. It has special effects on Dropout, BatchNorm, etc.
net.eval()

# Returns any extra state to include in the module’s state_dict.
# Implement this and a corresponding set_extra_state().param: target
target = "target_str"
net.get_extra_state(target)


# Returns the parameter given by target if it exists, otherwise throws an error.

net.get_parameter(target)

# Returns the submodule given by target if it exists, otherwise throws an error.
# assume you hav model: A.net_b.net_c.linear, to  To check whether or not we have
# the ``linear`` submodule, we would call ``get_submodule("net_b.linear")``
net.get_submodule(target)

# Copies parameters and buffers from state_dict into this module and its descendants
# state_dict includes parameters and buffers
net.load_state_dict(state_dict, strict=True)

# Returns an iterator over all modules in the network. Duplicate modules are returned only once.
net.modules()




In [39]:
net = Net()
# Additional information
EPOCH = 5
PATH = "model.pt"
LOSS = 0.4

torch.save({
    'epoch': EPOCH,
    'model_state_dict': net.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': LOSS,
}, PATH)

In [40]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()
# - or -
model.train()

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

# persistent = False, will not save, will not be a part of model's state_dict
# BatchNorm's ``running_mean`` is not a parameter
net.register_buffer(self, name: str, tensor: Optional[Tensor], persistent: bool = True)

# Parameter() is a subclass of Tensor, when Parameter is assigned as a module attribute,
# it will be auto added to the module's parameters, and appear in the iterator parameters()
# assigning a Tensor doesn't have such effect
net.register_parameter(self, name: str, param: Optional[Parameter])

In [42]:
import torch
import torch.nn as nn

class GaussianModel(nn.Module):
    def __init__(self):
        super(GaussianModel, self).__init__()
        self.mean = nn.Parameter(torch.zeros(1))
        self.pdf = torch.distributions.Normal(self.mean, torch.tensor([1.0]))

    def forward(self, x):
        return -self.pdf.log_prob(x)

In [45]:
# apply fn on all Parameters (._parameters) and Buffers (._buffers).
net._apply(init_weights)
net.cuda()

# Casts all parameters and buffers to
net.type()

Parameter containing:
tensor([[[[-0.0593,  0.0321, -0.0535,  0.0566, -0.0247],
          [ 0.0888,  0.0446, -0.0138,  0.0342,  0.0201],
          [-0.0408, -0.0749,  0.0199,  0.0353, -0.0504],
          [ 0.0173, -0.0933,  0.0903, -0.0871, -0.0575],
          [-0.0078, -0.1075,  0.0182,  0.0307,  0.0190]],

         [[ 0.0701, -0.0358,  0.0921,  0.0858,  0.0576],
          [ 0.1066,  0.0422, -0.1078,  0.0685,  0.1135],
          [-0.0934, -0.1065, -0.0767, -0.0419,  0.0700],
          [ 0.0784, -0.0037, -0.0501,  0.0086,  0.0525],
          [-0.0982,  0.0219, -0.0296,  0.0480, -0.0190]],

         [[ 0.0539,  0.0367,  0.1011, -0.0142, -0.0594],
          [ 0.0981, -0.1098,  0.0092, -0.0434,  0.1091],
          [ 0.0173,  0.0519,  0.0058, -0.0810, -0.0487],
          [ 0.0246, -0.0755,  0.1066,  0.0008,  0.0584],
          [-0.0262,  0.0325, -0.1141, -0.1060, -0.0720]]],


        [[[-0.0279,  0.0727,  0.0918, -0.0543, -0.0699],
          [-0.0298,  0.0843, -0.0487,  0.0128, -0.0811],
 

TypeError: _has_compatible_shallow_copy_type(): argument 'from' (position 2) must be Tensor, not NoneType

## part 8 - save checkpoint

In [5]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'test_torch_model.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /Users/zhou/.cache/torch/hub/checkpoints/vgg16-397923af.pth
15.1%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

24.0%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

29.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_

In [7]:
# without specifying pretrained=True, we can laod the weight later
model = models.vgg16()
model.load_state_dict(torch.load('test_torch_model.pth'))
# set the model to eval, has effect on some modules such as
# `Dropout`, `BatchNorm`, etc. Equal to model.train(False)
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### save the general checkpoint- Save model.state_dict and optimizer.state_dict

In [14]:
Epoch = 5
PATH = 'test_model.pth'
Loss = 0.4
torch.save({
    'epoch':Epoch,
    'model_state_dict':model.state_dict(),
    'optimizer_state_dict':optimizer.state_dict(),
    'loss':Loss
}, PATH)

## load the general checkpoint

In [15]:
model = Net()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.09)
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']
# has effect on `Dropout`, `BatchNorm`, etc
model.eval() # model.train(False)

RuntimeError: Error(s) in loading state_dict for Net:
	Missing key(s) in state_dict: "conv1.weight", "conv1.bias", "conv2.weight", "conv2.bias", "fc1.weight", "fc1.bias", "fc2.weight", "fc2.bias", "fc3.weight", "fc3.bias". 
	Unexpected key(s) in state_dict: "features.0.weight", "features.0.bias", "features.2.weight", "features.2.bias", "features.5.weight", "features.5.bias", "features.7.weight", "features.7.bias", "features.10.weight", "features.10.bias", "features.12.weight", "features.12.bias", "features.14.weight", "features.14.bias", "features.17.weight", "features.17.bias", "features.19.weight", "features.19.bias", "features.21.weight", "features.21.bias", "features.24.weight", "features.24.bias", "features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias", "classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias", "classifier.6.weight", "classifier.6.bias". 

In [16]:
class Test(nn.Module):
    """UDF class of module"""
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(2, 3)
        self.lienar2 = nn.Linear(3, 4)
        self.bach_norm = nn.BatchNorm2d(4)


test_model = Test()
test_model.modules

<bound method Module.modules of Test(
  (linear1): Linear(in_features=2, out_features=3, bias=True)
  (lienar2): Linear(in_features=3, out_features=4, bias=True)
  (bach_norm): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)>

In [17]:
test_model._modules

OrderedDict([('linear1', Linear(in_features=2, out_features=3, bias=True)),
             ('lienar2', Linear(in_features=3, out_features=4, bias=True)),
             ('bach_norm',
              BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True))])

In [19]:
test_model._modules['linear1'].weight
print(test_model._modules['linear1'].weight.dtype)

Parameter containing:
tensor([[-0.5020,  0.1869],
        [ 0.5380, -0.4350],
        [ 0.1956, -0.4930]], requires_grad=True)

In [21]:
test_model.to(torch.double)
print(test_model._modules['linear1'].weight.dtype)

torch.float64


### three important attr of a model, return the attr of current model
test_model._modules #
test_model._parameters # empty if not init parameters
test_model._buffers # empty if not init buffers

In [23]:
test_model.state_dict()

OrderedDict([('linear1.weight',
              tensor([[-0.5020,  0.1869],
                      [ 0.5380, -0.4350],
                      [ 0.1956, -0.4930]], dtype=torch.float64)),
             ('linear1.bias',
              tensor([ 0.0748,  0.4779, -0.3502], dtype=torch.float64)),
             ('lienar2.weight',
              tensor([[ 0.0116,  0.1853, -0.5164],
                      [ 0.0852,  0.1703,  0.1663],
                      [ 0.0964, -0.1226,  0.3712],
                      [-0.4571, -0.3320,  0.2632]], dtype=torch.float64)),
             ('lienar2.bias',
              tensor([ 0.4797,  0.2652, -0.3681,  0.4934], dtype=torch.float64)),
             ('bach_norm.weight',
              tensor([1., 1., 1., 1.], dtype=torch.float64)),
             ('bach_norm.bias', tensor([0., 0., 0., 0.], dtype=torch.float64)),
             ('bach_norm.running_mean',
              tensor([0., 0., 0., 0.], dtype=torch.float64)),
             ('bach_norm.running_var',
              tensor([1., 

In [45]:
class VerboseExecution(nn.Module):
    def __init__(self, model: nn.Module):
        super().__init__()
        self.model = model

        # Register a hook for each layer
        for name, layer in self.model.named_children():
            layer.__name__ = name
            layer.register_forward_hook(
                lambda layer, _, output: print(f"{layer.__name__}:out->{output.shape}")
            )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

In [46]:
resnet50 = models.resnet50()
verbose_resnet = VerboseExecution(resnet50)
dummy_input = torch.ones(10, 3, 224, 224)

_ = verbose_resnet(dummy_input)

conv1:out->torch.Size([10, 64, 112, 112])
bn1:out->torch.Size([10, 64, 112, 112])
relu:out->torch.Size([10, 64, 112, 112])
maxpool:out->torch.Size([10, 64, 56, 56])
layer1:out->torch.Size([10, 256, 56, 56])
layer2:out->torch.Size([10, 512, 28, 28])
layer3:out->torch.Size([10, 1024, 14, 14])
layer4:out->torch.Size([10, 2048, 7, 7])
avgpool:out->torch.Size([10, 2048, 1, 1])
fc:out->torch.Size([10, 1000])


In [50]:
from typing import Dict, Iterable, Callable

class FeatureExtractor(nn.Module):
    def __init__(self, model: nn.Module, layers: Iterable[str]):
        super().__init__()
        self.model = model
        self.layers = layers # ["layer4", "avgpool"]
        self._features = {layer: torch.empty(0) for layer in layers}

        for layer_id in layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            layer.register_forward_hook(self.save_outputs_hook(layer_id))

    def save_outputs_hook(self, layer_id: str) -> Callable:
        def fn(_, __, output):
            self._features[layer_id] = output
        return fn

    def forward(self, x: torch.Tensor) -> Dict[str, torch.Tensor]:
        _ = self.model(x)
        return self._features

In [63]:
vgg16 = models.vgg16()
print([component for component in vgg16.named_modules()])


[('', VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [66]:
resnet50.named_modules
resnet_features = FeatureExtractor(models.resnet50(), layers=["layer4", "avgpool"])
features = resnet_features(dummy_input)

print({name: output.shape for name, output in features.items()})

{'layer4': torch.Size([10, 2048, 7, 7]), 'avgpool': torch.Size([10, 2048, 1, 1])}


In [ ]:
def getStat(train_data):
    '''
    Compute mean and variance for training data
    :param train_data: 自定义类Dataset(或ImageFolder即可)
    :return: (mean, std)
    '''
    print('Compute mean and variance for training data.')
    print(len(train_data))
    train_loader = torch.utils.data.DataLoader(
        train_data, batch_size=1, shuffle=False, num_workers=0,
        pin_memory=True)
    mean = torch.zeros(3)
    std = torch.zeros(3)
    for X, _ in train_loader:
        for d in range(3):
            mean[d] += X[:, d, :, :].mean()
            std[d] += X[:, d, :, :].std()
    mean.div_(len(train_data))
    std.div_(len(train_data))
    return list(mean.numpy()), list(std.numpy())

if __name__ == '__main__':

    train_dataset = torchvision.datasets.ImageFolder(root=r'/home/zhou/datasets', transform=None)
    print(getStat(train_dataset))

In [94]:
for name, net_module in resnet50._modules.items():
    if name == 'layer1':
        for name1, net_module1 in net_module._modules.items():
            print(f'*****{name1}*****{net_module1 }*****')
    print(f'{name}->{net_module}')

conv1->Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
bn1->BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
relu->ReLU(inplace=True)
maxpool->MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
*****0*****Bottleneck(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, tr

In [104]:
vgg = models.vgg11()
for sub_module in vgg.children():
    print(sub_module,'***************\n***************\n')

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): ReLU(inplace=True)
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (16): Conv2d(512, 512, kernel_size=(3, 3), stride=

In [101]:
new_model = nn.Sequential(*list(vgg.children())[:-1])
print(new_model)

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [107]:
a_tensor = torch.ones(2,3)
print(a_tensor.)

['T', '__abs__', '__add__', '__and__', '__array__', '__array_priority__', '__array_wrap__', '__bool__', '__class__', '__complex__', '__contains__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__div__', '__dlpack__', '__dlpack_device__', '__doc__', '__eq__', '__float__', '__floordiv__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__iand__', '__idiv__', '__ifloordiv__', '__ilshift__', '__imod__', '__imul__', '__index__', '__init__', '__init_subclass__', '__int__', '__invert__', '__ior__', '__ipow__', '__irshift__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__long__', '__lshift__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rfloordiv__', '__rlshift__', '__rmatmul__', '__rmod__', '__rmul__', '

In [108]:
torch.backends.cudnn.benchmark

False

In [120]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.009)
optimizer.zero_grad()
optimizer.step()

In [117]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
optimizer.zero_grad()
optimizer.step()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda e:(e-100)**2/100**2, last_epoch=-1)

In [ ]:
torch.optim.lr_scheduler.LambdaLR
torch.nn.functional
torch.autograd.function
torch.autograd.grad_mode
torch.autograd.functional.jacobian
torch.nn.Conv2d
a_tensor.backward

In [121]:
class Exp(torch.autograd.Function):     # compute e^x of a layer
    @staticmethod
    def forward(ctx, i):
        result = i.exp()
        return result

@staticmethod
def backward(ctx, grad_output):
    # take out info saved in forward()
    result, = ctx.saved_tensors
    # compute grad
    return grad_output * result

x = torch.tensor([1.], requires_grad=True)
# use apply method to call the UDF, forward
ret = Exp.apply(x)
print(ret) #[2.7183] grad_fn=<ExpBackward>
ret.backward()
print(x.grad)   # tensor([2.7183])

KeyboardInterrupt: 

In [127]:
a = 257
b = 257
print(f'id a:{id(a)}, id b:{id(b)}')

id a:140521811550032, id b:140521811549680


In [128]:
str_a = 'a'
str_b = 'b'
print(f'id a:{id(str_a)}, id b:{id(str_b)}')

id a:140521532307824, id b:140521801289968


In [129]:
id('aa'*2) == id('a'*4)

True

In [130]:
net._modules

True

In [157]:
class Solution:
    def romanToInt(self, s: str) -> int:

        def sumUp(data):
            digit_map = {'I':1, 'V':5,'X':10, 'L':50, 'C':100, 'D':500, 'M':1000}
            res = 0
            for char in data:
                num = digit_map[char]
                res += num
            return res

        data = list(s)
        return sumUp(data)

In [158]:
Solution().romanToInt("MCMXCIV")

KeyboardInterrupt: 

In [148]:
list("III")

['I', 'I', 'I']

In [160]:
net.training

True

In [161]:
model_a = nn.Sequential(
    nn.Conv2d(1,20,5),
    nn.ReLU(),
    nn.Conv2d(20,64,5),
    nn.ReLU() )

In [164]:
model_b = nn.Sequential(OrderedDict([
    ('conv1', nn.Conv2d(1,20,5)),
    ('relu1', nn.ReLU()),
    ('conv2', nn.Conv2d(20,64,5)),
    ('relu2', nn.ReLU()) ]))

In [204]:
def timer(func):
    import time
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        time.sleep(1)
        long = time.time() - start
        print(f'it takes {long} seconds')
    print(wrapper)
    return wrapper

@timer
def add(a, b):
    return a + b

<function timer.<locals>.wrapper at 0x7fcda97d58c0>


In [228]:
 def repeat(num):
    print('enter the repeat')
    def my_decorator(func):
        print('enter my_decorator')
        def wrapper(*args, **kwargs):
            for i in range(num):
                print('wrapper of decorator')
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(3)
def greet(message):
    print(message)

greet('hello')

enter the repeat
enter my_decorator


KeyboardInterrupt: 

In [230]:
def test1(func):
    print("enter test1")

    def wrapper1(*args, **kwargs):
        print('wrapper1, before test1 ...')
        func(*args, **kwargs)
        print('wrapper1, after test1 ...')
    return wrapper1 #返回内层函数的引用

def test2(func):
    print("%s" %("enter test2"))

    def wrapper2(*args, **kwargs):
        print('wrapper2, before test2 ...')
        func(*args, **kwargs)
        print('wrapper2, after test2 ...')
    return wrapper2 #返回内层函数的引用

@test2
@test1
def add(a, b):
    print('execute original function', a+b)

add(1, 2) #正常调用add

enter test1
enter test2
wrapper2, before test2 ...
wrapper1, before test1 ...
execute original function 3
wrapper1, after test1 ...
wrapper2, after test2 ...


In [233]:
bb class Count:
    def __init__(self, func):
        self.func=func
        self.num_calls=0
    def __call__(self, *args, **kwargs):
        self.num_calls +=1
        return self.func(*args, **kwargs)

@Count
def example():
    print('hello')

example()

hello


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython_darwin_37_64.pyx", line 1035, in _pydevd_bundle.pydevd_cython_darwin_37_64.PyDBFrame.trace_dispatch
  File "/Applications/DataSpell.app/Contents/plugins/python-ce/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 144, in cmd_step_over
    if _is_inside_jupyter_cell(frame, pydb):
  File "/Applications/DataSpell.app/Contents/plugins/python-ce/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 209, in _is_inside_jupyter_cell
    if is_cell_filename(filename):
  File "/Applications/DataSpell.app/Contents/plugins/python-ce/helpers-pro/jupyter_debug/pydev_jupyter_plugin.py", line 220, in is_cell_filename
    ipython_shell = get_ipython()
NameError: name 'get_ipython' is not defined


In [234]:
class logger(object): # a class decorator with arguments
    def __init__(self, level='INFO'):
        self.level = level
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print('decoration is here')
            func(*args, **kwargs)
        return wrapper
@logger(level='WARNING')
def say(message):
    print(message)
say('hello, world!')

decoration is here
hello, world!


In [238]:
import time
import functools

class DelayFunc:
    def __init__(self, duration, func):
        self.duration = duration
        self.func = func
    def __call__(self, *args, **kwargs):
        print(f'waiting for {self.duration} seconds')
        time.sleep
        return self.func(*args, **kwargs)
    @property
    def eager_call(self, *args, **kwargs):
        return self.func(*args, **kwargs)

def delay(duration):
    return functools.partial(DelayFunc, duration)

@delay(duration=2)
def add(a,b):
    return a+b
add(2, 3)

waiting for 2 seconds


5

In [239]:
import functools

In [241]:
import itertools

In [248]:
def say_hello(self):
    print('hello')

In [249]:
class BaseClass:
    def talk(self):
        print('hello')
def say(self):
    print('hi, there')

    # use type to create User class
User = type('User', (BaseClass, ), {'name':'user', 'say':say_hello})


In [253]:
User().say()

hello


In [254]:
 class Fab(object):

    def __init__(self, max):
        self.max = max
        self.n, self.a, self.b = 0, 0, 1

    def __iter__(self):
        return self

    def next(self):
        if self.n < self.max:
            r = self.b
            self.a, self.b = self.b, self.a + self.b
            self.n = self.n + 1
            return r
        raise StopIteration()

In [256]:
fab = Fab(10)

In [263]:
class Animal:
    name = '老虎'
    unique = 0
    def __init__(self, name='老虎', age=5):
        self.name = name
        self.age = age
        self.weight = 200

    def eat(self):
        self.height = 100
        return '我需要吃东西！'

    @classmethod
    def sleep(cls):
        return '我需要睡觉'

class Dog(Animal):

    def __init__(self, age):
        self.age = age

a = Animal()
d = Dog(8)

# dict_keys(['__module__', 'name', 'unique', '__init__', 'eat', 'sleep', '__dict__', '__weakref__', '__doc__'])
print('class dict:',Animal.__dict__.keys())
print('inst dict:',a.__dict__.keys())
print(d.__dict__.keys())     # dict_keys(['age'])

class dict: dict_keys(['__module__', 'name', 'unique', '__init__', 'eat', 'sleep', '__dict__', '__weakref__', '__doc__'])
inst dict: dict_keys(['name', 'age', 'weight'])
dict_keys(['age'])


In [267]:
class Descriptor:
    def __init__(self, var1):
        print(f"descriptor init: {var1}")
        self.var1 = var1

    def __set__(self, instance, value):
        """"""
        print(f"Assigning value: {value}")

    def __get__(self, instance, owner=None):
        """"""
        print('Getting value')
        return "descriptor returned value"


class Person:

    foo = Descriptor("init")
    def __init__(self, value):
        self.foo = value

person = Person('instance') # descriptor init: init, Assigning instance
print("2:", person.foo)      # Getting value, 2 descriptor returned value
person.foo ='test'          # Assigning test
print("3:", person.foo)      # Getting value

descriptor init: init
Assigning value: instance
Getting value
2: descriptor returned value
Assigning value: test
Getting value
3: descriptor returned value


In [271]:
def foo():
    print("starting...")
    while True:
        res = yield 4
        print("res is:",res)
g = foo()
print(next(g))
print("*"*20)
print(next(g))

starting...
4
********************
res is: None
4


In [ ]:
import torch.nn as nn